In [318]:
from nltk.stem import PorterStemmer
import re
from nltk.corpus import stopwords
import nltk
from sklearn.metrics import pairwise_distances_argmin_min, pairwise_distances
from nltk.stem import WordNetLemmatizer
  

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\georg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [422]:
import urllib.request
import json as simplejson
import pprint
import sys
import pandas as pd
import numpy as np

In [342]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [319]:
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [320]:
def clean_string( text):
    # Lower string and strip whitespaces
    text = text.lower().strip()
    # keep only alpha numeric characters
    pattern = r'[^A-Za-z]+'
    text = re.sub(pattern, ' ', text)
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

In [347]:
# anish df
url = 'http://34.121.206.61:8983/solr/IRF22P1/select?indent=true&q.op=OR&q=*%3A*&rows=200000'
wt = "wt=json"
connection = urllib.request.urlopen(url)

if wt == "wt=json":
    response   = simplejson.load(connection) 
else:
    response   = eval(connection.read())
anish_df = pd.DataFrame(response['response']['docs'])

In [348]:
anish_df["topic"].value_counts()

Healthcare     3103
Technology     3004
Education      2961
Environment    2613
Politics       1774
Name: topic, dtype: int64

In [349]:
# shubankar df
url = 'http://34.125.178.234:8983/solr/IRF22P1/select?indent=true&q.op=OR&q=*%3A*&rows=200000'
wt = "wt=json"
connection = urllib.request.urlopen(url)

if wt == "wt=json":
    response   = simplejson.load(connection) 
else:
    response   = eval(connection.read())
shubankar_df = pd.DataFrame(response['response']['docs'])

In [350]:
shubankar_df["topic"].value_counts()

Education      9826
Politics       9756
Technology     9644
Environment    1094
Healthcare      482
Name: topic, dtype: int64

In [351]:
#baasit df
import pickle
objects = []
with (open("final_submission.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [352]:
baasit_df = objects[0]

In [353]:
baasit_df

,author,id,selftext,subreddit,title,full_link,topic,is_submission,created_at,body,parent_body,parent_id
0,Addy_Bishop,ubrebd,I understand what stock is but who creates it?...,explainlikeimfive,ELI5 where does stock come from?,https://www.reddit.com/r/explainlikeimfive/com...,,True,2022-04-25T18:07:22Z,,,
1,Ambitious_Category85,trsva6,,explainlikeimfive,ELI5: How do seasons exists? Isn't the earth's...,https://www.reddit.com/r/explainlikeimfive/com...,,True,2022-03-29T23:31:10Z,,,
2,twerkinmidget,trs358,"Switching the clocks seems excessive, it IS th...",explainlikeimfive,ELI5: I don’t understand the point of time zon...,https://www.reddit.com/r/explainlikeimfive/com...,,True,2022-03-29T22:52:06Z,,,
3,AlienRouge,trrkza,,explainlikeimfive,ELI5: what is Roko’s Basilisk?,https://www.reddit.com/r/explainlikeimfive/com...,,True,2022-03-29T22:31:22Z,,,
4,Not-Alpharious,trpcdk,Wouldn’t it be more of a survival advantage to...,explainlikeimfive,ELI5: Why do animals have mating seasons inste...,https://www.reddit.com/r/explainlikeimfive/com...,,True,2022-03-29T22:07:51Z,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
34934,Extension_Work_5185,i09byml,,indiasocial,,https://www.reddit.com/r/indiasocial/comments/...,,False,2022-03-11T16:45:01Z,"If you know how to ride a bicycle, riding a sc...",I'm a 17 y/o guy and have never driven a Scoty...,t3_tbtvtx
34935,aitchnyu,i09br75,,indiasocial,,https://www.reddit.com/r/indiasocial/comments/...,,False,2022-03-11T16:43:41Z,I used to ride in my housing colony and side r...,I'm a 17 y/o guy and have never driven a Scoty...,t3_tbtvtx
34936,this_is_Pranay,i09bo1d,,indiasocial,,https://www.reddit.com/r/indiasocial/comments/...,,False,2022-03-11T16:43:07Z,Just don't give full acceleration and it is ba...,I'm a 17 y/o guy and have never driven a Scoty...,t3_tbtvtx
34937,GoDFaTHeR_is,i09bm3k,,indiasocial,,https://www.reddit.com/r/indiasocial/comments/...,,False,2022-03-11T16:42:47Z,If you know how to ride a bicycle then you wil...,I'm a 17 y/o guy and have never driven a Scoty...,t3_tbtvtx


In [354]:
baasit_df["topic"].value_counts()

               31288
healthcare       796
environment      777
politics         711
technology       690
education        677
Name: topic, dtype: int64

In [428]:
df = pd.concat([anish_df, shubankar_df, baasit_df])
df['topic'] = df['topic'].str.lower()
df = df.replace(r'^\s*$', np.nan, regex=True)

In [429]:
df_ = df.dropna(subset=['topic', "selftext"])


In [356]:
healthcare_dataset = df[df["topic"] == "healthcare"]["selftext"]
environment_dataset = df[df["topic"] == "environment"]["selftext"]
politics_dataset = df[df["topic"] == "politics"]["selftext"]
technology_dataset = df[df["topic"] == "technology"]["selftext"]
education_dataset = df[df["topic"] == "education"]["selftext"]

In [357]:
healthcare_dataset_combined = healthcare_dataset.str.cat(sep=' ')
environment_dataset_combined = environment_dataset.str.cat(sep=' ')
politics_dataset_combined = politics_dataset.str.cat(sep=' ')
technology_dataset_combined = technology_dataset.str.cat(sep=' ')
education_dataset_combined = education_dataset.str.cat(sep=' ')

In [358]:

healthcare_dataset_combined_clean = clean_string(healthcare_dataset_combined)
environment_dataset_combined_clean = clean_string(environment_dataset_combined)
politics_dataset_combined_clean = clean_string(politics_dataset_combined)
technology_dataset_combined_clean = clean_string(technology_dataset_combined)
education_dataset_combined_clean = clean_string(education_dataset_combined)

In [359]:
corpus = [ healthcare_dataset_combined_clean, environment_dataset_combined_clean, politics_dataset_combined_clean, technology_dataset_combined_clean, education_dataset_combined_clean]


In [361]:
vectorizer = TfidfVectorizer()

# TD-IDF Matrix
X = vectorizer.fit_transform(corpus)
tfidf_tokens = vectorizer.get_feature_names()


In [362]:
import pandas as pd

result = pd.DataFrame(
    data=X.toarray(), 
    index=[ "healthcare", "environment", "politics", "technology", "education"], 
    columns=tfidf_tokens
)

In [363]:
result

,aa,aaa,aaaa,aaaaaa,aaaaaaa,aaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaand,aaaaaaaaapg,aaaaaaaaaw,...,zzvkb,zzwutk,zzxwkgtmxmbgm,zzyglatz,zzz,zzzmode,zzzs,zzzxxx,zzzz,zzzzingne
healthcare,0.000402,0.000050,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000013,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
environment,0.000327,0.000147,0.000000,0.000011,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000023,0.000000,0.000000,0.000000,0.000000,0.000000,0.000011,0.000011,0.000000
politics,0.000508,0.000021,0.000000,0.000000,0.000000,0.000044,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
technology,0.000661,0.000654,0.000014,0.000000,0.000007,0.000000,0.000007,0.000007,0.000007,0.000007,...,0.000000,0.000000,0.000000,0.000007,0.000007,0.000028,0.000007,0.000000,0.000000,0.000014
education,0.000362,0.000046,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [399]:
input_string = "The ambulance arrived within 20 minutes and she received attentive and professional care from the paramedics."
# input_string = "So how can you help? Join climate activists, work and volunteer for movements that protect ecology and cleanups, argue for degrowth"
# input_string = "Renewables will save the complex web of life on earth from climate disaster. They just can’t simultaneously save life on earth from all other man-made disasters."
# input_string = "Apple Makes Plans to Move Production Out of China"
# input_string = "So, I'm in my final semester of undergrad, and I am currently eligible to graduate. I'm taking four classes, three of which are required for me to finish my degree (passing these with flying colors) and one extra class I decided to take to fill some extra time. If I fail the extra class, which is not required for my major, am I still able to graduate"
# input_string = "I saw on the Canada subreddit that their healthcare system is crumbling."
input_string_clean = clean_string(input_string)

In [400]:
input_string_clean

'the ambulance arrived within minute and she received attentive and professional care from the paramedic'

In [401]:
input_string_vector = vectorizer.transform([input_string_clean])

In [402]:
_, pair_dist_arg = pairwise_distances_argmin_min(input_string_vector, result, metric='euclidean')

In [403]:
result.index[_[0]]

'technology'

In [404]:
pair_dist_matrix = pairwise_distances(input_string_vector, result, metric='euclidean')

In [405]:
pair_dist_matrix

array([[1.10530929, 1.06798157, 1.0651823 , 1.06029237, 1.0945927 ]])

In [413]:
df_

,author,body,id,parent_id,subreddit,created_at,is_submission,full_link,parent_body,_version_,title,selftext,topic
42632,anarchysquid,NaN,svtg1i,NaN,AskALiberal,2022-02-18T21:51:02Z,True,https://www.reddit.com/r/AskALiberal/comments/...,NaN,1.744522e+18,What's the best defense against Nutpicking?,"Nutpicking, for the uninitiated, is when the o...",politics
42633,TheCatholicsAreComin,NaN,svlnnn,NaN,neoliberal,2022-02-18T16:12:15Z,True,https://www.reddit.com/r/neoliberal/comments/s...,NaN,1.744522e+18,How would this sub teach racism in schools?,The culture war over how racism is or isn’t ta...,politics
42634,throw_away_2022_,NaN,svgwm4,NaN,northernireland,2022-02-18T12:34:02Z,True,https://www.reddit.com/r/northernireland/comme...,NaN,1.744522e+18,'New IRA' say they shot man in Tyrone,[https://www.irishnews.com/news/northernirelan...,politics
42635,SwimmingResearcher74,NaN,sv30hk,NaN,conspiracy,2022-02-17T23:45:55Z,True,https://www.reddit.com/r/conspiracy/comments/s...,NaN,1.744522e+18,Can someone explain why it’s okay for liberals...,Likewise why is it okay for them to freeze fun...,politics
42636,lotus_pond54,NaN,suy9ym,NaN,u_lotus_pond54,2022-02-17T20:20:45Z,True,https://www.reddit.com/r/u_lotus_pond54/commen...,NaN,1.744522e+18,Trump Terrorists are Stealing Me Blind of Litt...,They stole my harmonica so I can't sing with t...,politics
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34934,Extension_Work_5185,"If you know how to ride a bicycle, riding a sc...",i09byml,t3_tbtvtx,indiasocial,2022-03-11T16:45:01Z,False,https://www.reddit.com/r/indiasocial/comments/...,I'm a 17 y/o guy and have never driven a Scoty...,NaN,,,
34935,aitchnyu,I used to ride in my housing colony and side r...,i09br75,t3_tbtvtx,indiasocial,2022-03-11T16:43:41Z,False,https://www.reddit.com/r/indiasocial/comments/...,I'm a 17 y/o guy and have never driven a Scoty...,NaN,,,
34936,this_is_Pranay,Just don't give full acceleration and it is ba...,i09bo1d,t3_tbtvtx,indiasocial,2022-03-11T16:43:07Z,False,https://www.reddit.com/r/indiasocial/comments/...,I'm a 17 y/o guy and have never driven a Scoty...,NaN,,,
34937,GoDFaTHeR_is,If you know how to ride a bicycle then you wil...,i09bm3k,t3_tbtvtx,indiasocial,2022-03-11T16:42:47Z,False,https://www.reddit.com/r/indiasocial/comments/...,I'm a 17 y/o guy and have never driven a Scoty...,NaN,,,


In [430]:
len(df_)

29570

In [431]:
df_

,author,body,id,parent_id,subreddit,created_at,is_submission,full_link,parent_body,_version_,title,selftext,topic
42632,anarchysquid,NaN,svtg1i,NaN,AskALiberal,2022-02-18T21:51:02Z,True,https://www.reddit.com/r/AskALiberal/comments/...,NaN,1.744522e+18,What's the best defense against Nutpicking?,"Nutpicking, for the uninitiated, is when the o...",politics
42633,TheCatholicsAreComin,NaN,svlnnn,NaN,neoliberal,2022-02-18T16:12:15Z,True,https://www.reddit.com/r/neoliberal/comments/s...,NaN,1.744522e+18,How would this sub teach racism in schools?,The culture war over how racism is or isn’t ta...,politics
42634,throw_away_2022_,NaN,svgwm4,NaN,northernireland,2022-02-18T12:34:02Z,True,https://www.reddit.com/r/northernireland/comme...,NaN,1.744522e+18,'New IRA' say they shot man in Tyrone,[https://www.irishnews.com/news/northernirelan...,politics
42635,SwimmingResearcher74,NaN,sv30hk,NaN,conspiracy,2022-02-17T23:45:55Z,True,https://www.reddit.com/r/conspiracy/comments/s...,NaN,1.744522e+18,Can someone explain why it’s okay for liberals...,Likewise why is it okay for them to freeze fun...,politics
42636,lotus_pond54,NaN,suy9ym,NaN,u_lotus_pond54,2022-02-17T20:20:45Z,True,https://www.reddit.com/r/u_lotus_pond54/commen...,NaN,1.744522e+18,Trump Terrorists are Stealing Me Blind of Litt...,They stole my harmonica so I can't sing with t...,politics
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4801,SilverIsGoldenEye,NaN,tbu6fg,NaN,SaitamaInu_Official,2022-03-11T16:43:03Z,True,https://www.reddit.com/r/SaitamaInu_Official/c...,NaN,NaN,"Crazy or not, I believe in Saitama",They've got SO many things going on and while ...,education
4802,Martin-Physics,NaN,tbu1se,NaN,uvic,2022-03-11T16:36:55Z,True,https://www.reddit.com/r/uvic/comments/tbu1se/...,NaN,NaN,Let's talk about masking...,Yet another post on masking! Oh no!\n\nI think...,education
4803,Lost_Ad2633,NaN,tbu0gb,NaN,relationship_advice,2022-03-11T16:35:09Z,True,https://www.reddit.com/r/relationship_advice/c...,NaN,NaN,I [28 M] was broken up with by my Girlfriend [...,Long post incoming but I feel like there is a ...,education
4804,Giana95,NaN,tbtyp3,NaN,WorldofTanks,2022-03-11T16:32:44Z,True,https://www.reddit.com/r/WorldofTanks/comments...,NaN,NaN,I have 21% win rate over the last 150 matches....,Huge rant incoming.\n\n&amp;#x200B;\n\nI start...,education


In [432]:
df_sample = df_.sample(n = 1000)


In [437]:
def predict_topic(input_string):
    input_string_clean = clean_string(input_string)
    input_string_vector = vectorizer.transform([input_string_clean])
    _, pair_dist_arg = pairwise_distances_argmin_min(input_string_vector, result, metric='euclidean')
    return result.index[_[0]]

In [ ]:
df_["predicted_topic"] = df_["selftext"].apply(predict_topic)

In [438]:
df_sample["predicted_topic"] = df_sample["selftext"].apply(predict_topic)

In [439]:
df_sample

,author,body,id,parent_id,subreddit,created_at,is_submission,full_link,parent_body,_version_,title,selftext,topic,predicted_topic
46744,khaleyi,NaN,t3plch,NaN,QuebecLibre,2022-02-28T20:51:33Z,True,https://www.reddit.com/r/QuebecLibre/comments/...,NaN,1.744522e+18,/rQuébecLibre vs /rQuébec,Bonjour ca fait pas très longtemps que je suis...,healthcare,healthcare
44865,slipperyslevine,NaN,l0uhxd,NaN,wallstreetbets,2021-01-19T22:04:56Z,True,https://www.reddit.com/r/wallstreetbets/commen...,NaN,1.744522e+18,$BB TRUST IN JOHN CHEN,TLDR: This man DOES NOT KNOW HOW TO FUCKING LO...,technology,politics
47288,xDawnut,NaN,6tl9ju,NaN,New upcoming Google Calendar desktop UI,2017-08-14T09:10:12Z,True,1.502701812E9,NaN,1.744559e+18,http://imgur.com/a/xSiE7,google,technology,healthcare
56838,sleepyblue21,NaN,lxfg57,NaN,college,2021-03-04T07:45:04Z,True,https://www.reddit.com/r/college/comments/lxfg...,NaN,1.744559e+18,Would taking a gap year before college be bad?,I’m trying to decide what i want to do after h...,education,education
56818,fleur-escent,NaN,t558jw,NaN,Teachers,2022-03-02T17:25:56Z,True,https://www.reddit.com/r/Teachers/comments/t55...,NaN,1.744539e+18,Tired of Kids Doing NOTHING and expecting to pass,"So I have some of the most entitled, arrogant ...",education,education
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52664,Gainzz1011,NaN,otoe3n,NaN,FIREIndia,2021-07-29T02:12:47Z,True,https://www.reddit.com/r/FIREIndia/comments/ot...,NaN,1.744522e+18,Fat (obese?) Fire amount in India,"Hi I will keep it brief. I am in my late 30s, ...",environment,education
56477,Crazy_Cry_1257,NaN,mnaq49,NaN,college,2021-04-09T05:51:52Z,True,https://www.reddit.com/r/college/comments/mnaq...,NaN,1.744559e+18,Is it possible/realistic to take 10 classes?,I’m really hating college and I’m just ready t...,education,education
52560,Figurative_Retard,NaN,qw7gvf,NaN,college,2021-11-17T19:54:25Z,True,https://www.reddit.com/r/college/comments/qw7g...,NaN,1.744559e+18,"What's the craziest thing, academically, you'v...",How did you defy the odds? How did you manage ...,education,technology
51510,AutoModerator,NaN,eigz8p,NaN,loseit,2020-01-01T10:24:44Z,True,https://www.reddit.com/r/loseit/comments/eigz8...,NaN,1.744522e+18,Day 1? Starting your weight loss journey on We...,**Today is your Day 1?**\n\n**Welcome to r/Los...,environment,environment


In [440]:
df_sample["predicted_topic"].value_counts()

education      537
politics       214
technology      98
environment     92
healthcare      59
Name: predicted_topic, dtype: int64

In [442]:
df_sample["label"] = df_sample["topic"] == df_sample["predicted_topic"] 

In [443]:
df_sample["label"].value_counts()

False    506
True     494
Name: label, dtype: int64